<a href="https://colab.research.google.com/github/gwon0919/ML/blob/main/tfc32grapheme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# token이 자소 단위인 데이터로 자연어 생성 모델 작성
!pip install jamotools

In [2]:
import jamotools
import tensorflow as tf
import numpy as np
import keras

path = keras.utils.get_file(
    'rnn_test_toji.txt',
    origin='https://raw.githubusercontent.com/pykwon/etc/master/rnn_test_toji.txt')
text = open(path,'rb').read().decode(encoding='utf-8')
print('전체 글자 수 length of text : {}'.format(len(text)))
print(text[:100])

1588545/1588545 [==============================] - 0s 0us/step
전체 글자 수 length of text : 695685
제 1 편 어둠의 발소리
1897년의 한가위.
까치들이 울타리 안 감나무에 와서 아침 인사를 하기도 전에, 무색 옷에 댕기꼬리를 늘인 
아이들은 송편을 입에 물고 마을길을 쏘


In [3]:
# 한글 자료 자모 단위로 분리. 한자에는 영향이 없다.
# s_split = jamotools.split_syllables(text[:100])
# print(s_split)
# rever = jamotools.join_jamos(s_split)
# print(rever)
# print(text[:50] == rever)

train_text_x = jamotools.split_syllables(text)
# 단어 사전
vocab = sorted(set(train_text_x))
vocab.append('UNK')
print('{} unique words'.format(len(vocab)))   # 179 unique words

char2idx = { w:i for i, w in  enumerate(vocab)}
idx2char = np.array(vocab)
print(idx2char)        # ['\n' '\r' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '/' '0'...
text_as_int = np.array([char2idx[c] for c in train_text_x])
print(text_as_int)     # [69 81  2 ...  2  1  0]
print(len(text_as_int))    # 1345233
print(train_text_x[:20])   # ㅈㅔ 1 ㅍㅕㄴ ㅇㅓㄷㅜㅁㅇㅢ ㅂㅏㄹ
print(text_as_int[:20])    # [69 81  2 13  2 74 82 49  2 68 80 52 89 62 68 95  2 63 76 54]

179 unique words
['\n' '\r' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '/' '0' '1' '2' '3' '4' '5'
 '6' '7' '8' '9' ':' '?' 'M' 'X' 'a' 'c' 'd' 'f' 'k' 'l' 'n' 'o' 'p' 'r'
 't' 'w' '×' '―' '‘' '’' '“' '”' '…' '\u3000' 'ㄱ' 'ㄲ' 'ㄳ' 'ㄴ' 'ㄵ' 'ㄶ' 'ㄷ'
 'ㄸ' 'ㄹ' 'ㄺ' 'ㄻ' 'ㄼ' 'ㄽ' 'ㄾ' 'ㄿ' 'ㅀ' 'ㅁ' 'ㅂ' 'ㅃ' 'ㅄ' 'ㅅ' 'ㅆ' 'ㅇ' 'ㅈ' 'ㅉ'
 'ㅊ' 'ㅋ' 'ㅌ' 'ㅍ' 'ㅎ' 'ㅏ' 'ㅐ' 'ㅑ' 'ㅒ' 'ㅓ' 'ㅔ' 'ㅕ' 'ㅖ' 'ㅗ' 'ㅘ' 'ㅙ' 'ㅚ' 'ㅛ'
 'ㅜ' 'ㅝ' 'ㅞ' 'ㅟ' 'ㅠ' 'ㅡ' 'ㅢ' 'ㅣ' '一' '下' '主' '事' '亡' '人' '代' '佛' '刑' '割'
 '化' '匠' '千' '呪' '啓' '善' '四' '地' '壁' '壽' '天' '太' '妄' '婚' '子' '孫' '寺' '山'
 '工' '布' '常' '平' '年' '役' '性' '情' '惡' '意' '方' '日' '春' '杖' '榮' '母' '氣' '水'
 '池' '淨' '無' '燈' '父' '眞' '示' '祈' '祭' '私' '童' '竹' '笠' '籍' '精' '絶' '置' '者'
 '萬' '術' '衣' '谷' '身' '迷' '造' '銀' '錫' '長' '陷' '電' '順' '食' '金' '落' '來' 'UNK']
[69 81  2 ...  2  1  0]
1345233
ㅈㅔ 1 ㅍㅕㄴ ㅇㅓㄷㅜㅁㅇㅢ ㅂㅏㄹ
[69 81  2 13  2 74 82 49  2 68 80 52 89 62 68 95  2 63 76 54]


In [4]:
# dataset 작성
seq_length = 80    # 80개의 자모가 주어질 경우 다음 자모를 예측
example_per_epoch = len(text_as_int) // seq_length
print(example_per_epoch)


char_dataset= tf.data.Dataset.from_tensor_slices(text_as_int)  # 전체가 아니라 부분 데이터 읽기
char_dataset = char_dataset.batch(seq_length + 1, drop_remainder=True)  # drop_remainder=True : 마지막 배치 크기를 무시
# seq_length + 1 : 처음 25개 단어(feature)와 그 뒤에 나오는 정답(label)이될 한 단어를 합쳐 반환하기 위함

for item in char_dataset.take(1):     # batch를 한 번씩 불러옴
  print(item.numpy())                   # [43654  6 50166 34431    21971     0
  print(idx2char[item.numpy()])         # ['제' '1' '편' '어둠의' '발소리' '\n'

print()
def split_input_target(chunk):
  return [chunk[:-1], chunk[-1]]       # [25단어], [1단어]

train_dataset = char_dataset.map(split_input_target)
for x, y in train_dataset.take(1):
  print(idx2char[x.numpy()])
  print(x.numpy())
  print(idx2char[y.numpy()])
  print(y.numpy())

16815
[69 81  2 13  2 74 82 49  2 68 80 52 89 62 68 95  2 63 76 54 66 84 54 96
  1  0 13 20 21 19 49 82 49 68 95  2 75 76 49 46 76 68 92 10  1  0 47 76
 71 96 52 94 54 68 96  2 68 89 54 73 76 54 96  2 68 76 49  2 46 76 62 49
 76 62 89 68 81  2 68 85 66]
['ㅈ' 'ㅔ' ' ' '1' ' ' 'ㅍ' 'ㅕ' 'ㄴ' ' ' 'ㅇ' 'ㅓ' 'ㄷ' 'ㅜ' 'ㅁ' 'ㅇ' 'ㅢ' ' ' 'ㅂ'
 'ㅏ' 'ㄹ' 'ㅅ' 'ㅗ' 'ㄹ' 'ㅣ' '\r' '\n' '1' '8' '9' '7' 'ㄴ' 'ㅕ' 'ㄴ' 'ㅇ' 'ㅢ' ' '
 'ㅎ' 'ㅏ' 'ㄴ' 'ㄱ' 'ㅏ' 'ㅇ' 'ㅟ' '.' '\r' '\n' 'ㄲ' 'ㅏ' 'ㅊ' 'ㅣ' 'ㄷ' 'ㅡ' 'ㄹ' 'ㅇ'
 'ㅣ' ' ' 'ㅇ' 'ㅜ' 'ㄹ' 'ㅌ' 'ㅏ' 'ㄹ' 'ㅣ' ' ' 'ㅇ' 'ㅏ' 'ㄴ' ' ' 'ㄱ' 'ㅏ' 'ㅁ' 'ㄴ'
 'ㅏ' 'ㅁ' 'ㅜ' 'ㅇ' 'ㅔ' ' ' 'ㅇ' 'ㅘ' 'ㅅ']

['ㅈ' 'ㅔ' ' ' '1' ' ' 'ㅍ' 'ㅕ' 'ㄴ' ' ' 'ㅇ' 'ㅓ' 'ㄷ' 'ㅜ' 'ㅁ' 'ㅇ' 'ㅢ' ' ' 'ㅂ'
 'ㅏ' 'ㄹ' 'ㅅ' 'ㅗ' 'ㄹ' 'ㅣ' '\r' '\n' '1' '8' '9' '7' 'ㄴ' 'ㅕ' 'ㄴ' 'ㅇ' 'ㅢ' ' '
 'ㅎ' 'ㅏ' 'ㄴ' 'ㄱ' 'ㅏ' 'ㅇ' 'ㅟ' '.' '\r' '\n' 'ㄲ' 'ㅏ' 'ㅊ' 'ㅣ' 'ㄷ' 'ㅡ' 'ㄹ' 'ㅇ'
 'ㅣ' ' ' 'ㅇ' 'ㅜ' 'ㄹ' 'ㅌ' 'ㅏ' 'ㄹ' 'ㅣ' ' ' 'ㅇ' 'ㅏ' 'ㄴ' ' ' 'ㄱ' 'ㅏ' 'ㅁ' 'ㄴ'
 'ㅏ' 'ㅁ' 'ㅜ' 'ㅇ' 'ㅔ' ' ' 'ㅇ' 'ㅘ']
[69 81  2 13  2 74 82 49  2 68 80 52 89 62 68 95  2 63 76 54 66 84 54 96
  1  0 1

In [5]:
# model
BATCH_SIZE = 64
steps_per_epoch = example_per_epoch // BATCH_SIZE
BUFFER_SIZE = 5000

# shuffle을 사용하면 epoch 마다 Dataset을 섞을 수 있다. 과적합 방지에 효과적
train_dataset = train_dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

total_char = len(vocab)
print(total_char)  # 179

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_char, 100, input_length=seq_length), # 밀집벡터, 100차원으로
    tf.keras.layers.LSTM(units=256, return_sequences=True),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.LSTM(units=256),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=total_char, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

179
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 100)           17900     
                                                                 
 lstm (LSTM)                 (None, 80, 256)           365568    
                                                                 
 dropout (Dropout)           (None, 80, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 179)               46003     
                                                                 
Total params: 954783 (3.64 MB)
Trainable params: 954

In [6]:
# 단어 단위 생성 모델 학습
from keras.preprocessing.sequence import pad_sequences

def testmodelFunc(epoch, logs):
  if epoch % 5 != 0 and epoch !=49:   #  5의 배수 이거나 49이면 처리
    return

  test_sentence = text[:48]
  test_sentence = jamotools.split_syllables(test_sentence)

  next_char = 300
  for _ in range(next_char):
    test_text_x = test_sentence[-seq_length:]
    test_text_x = np.array([char2idx[c] if c in char2idx else char2idx['UNK'] for c in test_text_x])
    test_text_x = pad_sequences([test_text_x], maxlen=seq_length, padding='pre', value=char2idx['UNK'])
    output_idx = np.argmax(model.predict(test_text_x)[0])  # 출력값 중에서 가장 값이 큰 인덱스 반환
    test_sentence += idx2char[output_idx]  # 출력 단어는 test_sentence에 누적해 다음 작업의 입력으로 활용

  print()
  print(jamotools.join_jamos(test_sentence))
  print()

# epoch이 끝날 때 마다 testmodelFunc를 해 진행 결과를 출력.
# fit 할 때(학습 도중) 학습 데이터가 predict 되는 과정을 확인해가며 작업하고 싶을 때사용
testModelCb = tf.keras.callbacks.LambdaCallback(on_epoch_begin=testmodelFunc)

# repeat() : input을 반복, 1개의 에폭의 끝과 다음 에폭의 시작에 상관없이 인자 만큼 반복함
history = model.fit(train_dataset.repeat(), epochs=50,
                    steps_per_epoch = steps_per_epoch,   # 한 에폭에 사용할 step 수를 지정. ex) 총 45개 sample이 있고 배치사이즈가 3이라면 15스텝으로 지정
                   callbacks =[testModelCb], verbose=2)


1/1 [==============================] - 0s 19ms/step

제 1 편 어둠의 발소리
1897년의 한가위.
까치들이 울타리 안 감나무에 와서 아妄妄妄常常常常常常常oonnnnnn化化化化化化o代代代代代””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””

Epoch 1/50
262/262 - 14s - loss: 3.2124 - accuracy: 0.1273 - 14s/epoch - 52ms/step
Epoch 2/50
262/262 - 4s - loss: 2.5643 - accuracy: 0.2529 - 4s/epoch - 17ms/step
Epoch 3/50
262/262 - 6s - loss: 2.3691 - accuracy: 0.2964 - 6s/epoch - 23ms/step
Epoch 4/50
262/262 - 4s - loss: 2.2902 - accuracy: 0.3109 - 4s/epoch - 16ms/step
Epoch 5/50
262/262 - 4s - loss: 2.2278 - accuracy: 0.3257 - 4s/epoch - 17ms/step
1/1 [==============================] - 0s 20ms/step

제 1 편 어둠의 발소리
1897년의 한가위.
까치들이 울타리 안 감나무에 와서 알을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼을 얼

In [8]:
print(history.history['loss'][-1])
print(history.history['accuracy'][-1])

model.save('tfc32model.hfd5')

0.9889540076255798
0.6763477921485901


In [9]:
from keras.models import load_model
model = load_model('tfc32model.hfd5')

# 임의의 문장을 사용해 생성된 새로운 문장 확인
test_sentence = "이날은 수수개비를 꺾어도 아이들은 매를 맞지 않는다"

next_char = 100

for _ in range(next_char):
    test_text_x = test_sentence[-seq_length:]
    test_text_x = np.array([char2idx[c] if c in char2idx else char2idx['UNK'] for c in test_text_x])
    test_text_x = pad_sequences([test_text_x], maxlen=seq_length, padding='pre', value=char2idx['UNK'])
    output_idx = np.argmax(model.predict(test_text_x)[0])  # 출력값 중에서 가장 값이 큰 인덱스 반환
    test_sentence += idx2char[output_idx]  # 출력 단어는 test_sentence에 누적해 다음 작업의 입력으로 활용


print(jamotools.join_jamos(test_sentence))


1/1 [==============================] - 0s 21ms/step
이날은 수수개비를 꺾어도 아이들은 매를 맞지 않는다길이는 것이다. 그러나 있는데 것대진다. 거선은 어던 한을 버리를 쌀았다. 그러나 감긴ㄷ
